## Análisis de Simulated Annealing para una fuerza de ventas de 5 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax
+ Tmin 
+ steps 
+ updates 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 5 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 10 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 5 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [2]:
# Librerías

import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [3]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [4]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'Tmax': {10000, 25000},
              'Tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. 

En este caso se considerarán 3 valores de *updates*, 2 valores del *steps*, 2 valores de *Tmax* y 2 de *Tmin* dando lugar a un total de 36 combinaciones. 

Cada combinación de hiperarámetros se correrá 10 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 10 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [17]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    15.05%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    16.54%     0.05%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.30%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.53%     0.08%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    18.20%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.44%     0.00%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    17.20%     0.00%     0:00:02    -1:59:59 Temperature 

     0.01100          5.60    18.10%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    17.53%     0.05%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    15.25%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.15%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.20%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.56%     0.10%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.60%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870 

     0.01300          5.60    17.75%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.13%     0.16%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.80%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.51%     0.18%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    18.05%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    16.53%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    17.25%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000 

     0.00870          5.60    15.70%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.68%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.10%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.86%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    15.85%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    17.11%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    15.45%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100 

     0.00870          5.60    15.75%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    17.00%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01700          5.60    14.75%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01700          5.60    17.33%     0.50%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580          5.60    15.75%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580          5.60    16.78%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.40%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300 

     0.00730          5.60    16.55%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    16.61%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    16.10%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    16.59%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    16.30%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    17.42%     0.04%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.75%     0.00%     0:00:03    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300 

     0.00730          5.60    18.15%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    16.77%     0.04%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580          5.60    16.00%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580          5.60    16.84%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.70%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.88%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    14.75%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730 

CPU times: user 52min 27s, sys: 17.1 s, total: 52min 44s
Wall time: 52min 55s


In [18]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 100}",5.604,5.604,10/10
1,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 10}",5.604,5.604,10/10
2,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 50}",5.604,5.604,10/10
3,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 100}",5.604,5.604,10/10
4,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 10}",5.604,5.604,10/10
5,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 50}",5.604,5.604,10/10
6,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 100}",5.604,5.604,10/10
7,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 10}",5.604,5.604,10/10
8,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 50}",5.604,5.604,10/10
9,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 500, 'updates': 100}",5.604,5.604,10/10


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $10$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que en ninguna combinación de parámetros hubo variación en la distancia minima. 

+ Sobre el resto de los parámetros, se escogieron valores alrededor e incluyendo los parámetros por omisión del algoritmo. 

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 10 veces, dando un total de 360 corridas. En las 10 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima y máxima obtenida: $4 km$ es la misma para las 36 combinaciones. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo da los mismos resultados. 
 
+ Estos resultados pueden ser consecuencia de haber utilizando los parámetros por default, por eso se hizo el siguiente experimento: correr el algoritmo 10 veces solo con 1 paso, updates con valor 2. Tmin con valor 1 y Tmax con valor 2. Fue interesante que la distancia minima y máxima de nuevo fue $4 km$. Puede que la estabilidad sea resultado del número pequeño de nodos. 


In [5]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

dict_Hiper_SA = {'Tmax': {2},
              'Tmin': {1},
              'steps': {1},
              'updates': {2}
              }


df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    17.00%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    33.91%     8.58%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.45%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.76%     6.98%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    17.80%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    31.72%     7.59%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    18.50%     0.00%     0:00:02    -1:59:58 Temperature 

CPU times: user 1min 21s, sys: 227 ms, total: 1min 21s
Wall time: 1min 22s


In [6]:
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 2, 'Tmin': 1, 'steps': 1, 'updates': 2}",5.604,5.604,10/10


### 2. GridSearch con parametros simples

In [7]:
rutas = pd.DataFrame(index=range(100),columns=['km', 'Ruta'])


dict_Hiper_SA = {'Tmax': 2,
              'Tmin': 1,
              'steps': 1,
              'updates': 2
              }


In [8]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,NaN
98,NaN,NaN


In [9]:
for corrida in range(100):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta =  ut.convert(SA.lst_MejorCamino)
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    16.65%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    28.67%     5.99%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    16.95%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    30.76%     7.06%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.40%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    33.85%     8.57%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    14.85%     0.00%     0:00:02    -1:59:58 Temperature 

     0.00870          5.60    17.20%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.92%     7.12%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    15.75%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    31.80%     7.55%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.60%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.63%     6.96%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    16.25%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870 

     0.01100          5.60    16.60%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01100          5.60    31.60%     7.48%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    15.95%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.94%     7.09%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    15.45%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00730          5.60    30.22%     6.83%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300          5.60    16.55%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01300 

     0.01500          5.60    17.05%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01500          5.60    35.23%     9.29%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    17.35%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.81%     7.11%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    14.95%     0.00%     0:00:02    -1:59:58 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870          5.60    30.65%     7.01%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000          5.60    17.70%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000 

In [13]:
# Primeras 7 rutas obtenidas
rutas.head(7)

,km,Ruta
0,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
1,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
2,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
3,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
4,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
5,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"
6,5.604,"[11037, 1020235635, 1020053072, 1006681965, 1001402004, 1020402992]"


In [15]:
# Descripción de las rutas obtenidas

rutas.groupby('Ruta').describe()

km         \
                                                              count unique   
Ruta                                                                         
11037-1020235635-1020053072-1006681965-1001402004-1020402992  100.0    1.0   

                                                                            
                                                                top   freq  
Ruta                                                                        
11037-1020235635-1020053072-1006681965-1001402004-1020402992  5.604  100.0

### Conclusiones:
+ Los resultados del algoritmo son estables.